---
title: "Spotify Project"
subtitle: "Numeric prediction model"
author: "Oleg Ryzhkov (oryzhkov@chicagobooth.edu)"
date: March 13, 2021
output: neural network model numeric prediction 
---

Load the H2O R package.

- R installation instructions are at http://h2o.ai/download  

In [2]:
library(h2o)

In [3]:
# Start up a 1-node H2O server on your local machine, 
# and allow it to use all CPU cores and up to 8GB of memory.

h2o.init(nthreads=-1, max_mem_size="8G")
h2o.no_progress()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         7 days 12 hours 
    H2O cluster timezone:       America/Chicago 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.32.0.1 
    H2O cluster version age:    5 months and 4 days !!! 
    H2O cluster name:           H2O_started_from_R_olegryzhkov_tpd759 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.71 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  1 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4 
    R Version:                  R version 4.0.2 (2020-06-22) 


Warning message in h2o.clusterInfo():
“
Your H2O cluster version is too old (5 months and 4 days)!
Please download and install the latest version from http://h2o.ai/download/”


The `h2o.deeplearning` function fits H2O's 
Deep Learning models from within R.
```{r}
help(h2o.deeplearning)
```

While H2O Deep Learning has many parameters, it is quite easy to use.
Most often we can use the default parameters for many of the inputs.
We should change the following parameters:

- the number and sizes of hidden layers;
- the number of epochs and the activation function;
- add some regularization technique.
  
  
We will apply it to `target_numeric_popularity` data, which
we got cleaning and transforming the original dataset.

In [24]:
target.df <- read.csv('../data/target_numeric_popularity.csv')
head(target.df)

,popularity,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,⋯,language.english,language.catalan,language.german,language.frisian,language.welsh,artist.number,artist.rank_sum,artist.trackcount,artist.trackrankedcount,artist.rank_mean
,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,55,0.394,0.484,160187,0.265,0,0.00e+00,0,0.1490,-11.101,⋯,0,0,0,0,0,1,2143,183,182,11.77
2,39,0.255,0.405,141987,0.365,0,4.68e-06,6,0.1630,-10.226,⋯,0,0,0,0,0,1,2143,183,182,11.77
3,34,0.406,0.477,157093,0.352,0,0.00e+00,1,0.1220,-14.165,⋯,1,0,0,0,0,1,2143,183,182,11.77
4,27,0.887,0.319,187333,0.201,0,0.00e+00,7,0.9040,-17.796,⋯,1,0,0,0,0,1,22115,773,773,28.61
5,26,0.938,0.269,236800,0.129,0,4.87e-06,7,0.6830,-18.168,⋯,0,0,0,0,0,1,22115,773,773,28.61
6,30,0.360,0.617,114987,0.711,0,1.59e-06,9,0.0841,-6.433,⋯,0,0,0,0,0,1,2143,183,182,11.77


In [33]:
# download the file if it does not exist
# if (!file.exists("target_numeric_popularity.csv"))
#     download.file('https://raw.githubusercontent.com/OlegRyzhkov2020/SpotifyPopularity/main/data/target_numeric_popularity.csv', 'target_numeric_popularity.csv')

target.df <- h2o.importFile('target.csv') 
dim(target.df)
head(target.df)


ERROR: Unexpected HTTP Status code: 404 Not Found (url = http://localhost:54321/3/ImportFiles?path=target.csv&pattern=)

water.exceptions.H2ONotFoundArgumentException
 [1] "water.exceptions.H2ONotFoundArgumentException: File target.csv does not exist"                               
 [2] "    water.persist.PersistNFS.importFiles(PersistNFS.java:127)"                                               
 [3] "    water.persist.PersistManager.importFiles(PersistManager.java:386)"                                       
 [4] "    water.api.ImportFilesHandler.importFiles(ImportFilesHandler.java:25)"                                    
 [5] "    jdk.internal.reflect.GeneratedMethodAccessor16.invoke(Unknown Source)"                                   
 [6] "    java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)"
 [7] "    java.base/java.lang.reflect.Method.invoke(Method.java:567)"                                              
 [8] "    water.api.

ERROR: Error in .h2o.doSafeREST(h2oRestApiVersion = h2oRestApiVersion, urlSuffix = page, : 

ERROR MESSAGE:

File target.csv does not exist




In [30]:
# download the file if it does not exist
if (!file.exists("UsedCars.csv"))
  download.file('https://raw.githubusercontent.com/ChicagoBoothML/MLClassData/master/UsedCars/UsedCars.csv', 'UsedCars.csv')

df <- h2o.importFile("UsedCars.csv") 
dim( df )
df

[1] 20063    11

  price trim isOneOwner mileage year  color displacement     fuel region
1  2988  320          f  193296 1995  Black          3.2 Gasoline    SoA
2  6595  320          f  129948 1995  other          3.2 Gasoline    Mid
3  7993  320          f  140428 1997  White          3.2 Gasoline    Mid
4  5995  420          f  113622 1999 Silver          4.2 Gasoline    Mid
5  3000  420          f  167673 1999 Silver          4.2 Gasoline    SoA
6  7400  430          f   82419 2002  White          4.3 Gasoline    Mid
  soundSystem wheelType
1        unsp     Alloy
2     Premium     Alloy
3        Bose     Alloy
4        unsp     Alloy
5        unsp     Alloy
6        Bose     Alloy

[20063 rows x 11 columns] 

function (name, pos = -1L, envir = as.environment(pos), all.names = FALSE, 
    pattern, sorted = TRUE) 
{
    if (!missing(name)) {
        pos <- tryCatch(name, error = function(e) e)
        if (inherits(pos, "error")) {
            name <- substitute(name)
            if (!is.character(name)) 
                name <- deparse(name)
            warning(gettextf("%s converted to character string", 
                sQuote(name)), domain = NA)
            pos <- name
        }
    }
    all.names <- .Internal(ls(envir, all.names, sorted))
    if (!missing(pattern)) {
        if ((ll <- length(grep("[", pattern, fixed = TRUE))) && 
            ll != length(grep("]", pattern, fixed = TRUE))) {
            if (pattern == "[") {
                pattern <- "\\["
                warning("replaced regular expression pattern '[' by  '\\\\['")
            }
            else if (length(grep("[^\\\\]\\[<-", pattern))) {
                pattern <- sub("\\[<-", "\\\\\\[<-", pattern)
                warning("replaced '[<-' by '\\\\[<-' in regular expression pattern")
            }
        }
        grep(pattern, all.names, value = TRUE)
    }
    else all.names
}
<bytecode: 0x7fc08e508ce8>
<environment: namespace:base>